# 1880-2010年间全美婴儿姓名  
pandas dataframe pd.read_csv()  
python 字符串插值 数字  
pandas dataframe pd.concat()  
pandas dataframe d.pivot_table()  
pandas dataframe d.tail()  
pandas dataframe d.plot()  
pandas DataFrameGroupBy d.apply(func)  
pandas dataframe d.xxx  
pandas series s.astype()  
pandas series s.sum()  
pandas dataframe d.sort_values()  
pandas dataframe d.drop()  
numpy 逻辑函数 n.allclose() 

In [ ]:
import pandas as pd
names1880 = pd.read_csv('ch02/yob1880.txt',names=['name','sex','births'],header=None)
names1880[:10]

In [ ]:
names1880.info()

In [ ]:
#获取按sex分组统计
names1880.groupby('sex').births.sum()

In [ ]:
#将各个年份的数据合并
years = range(1880,2011)

pieces = []
columns = ['name','sex','births']

for year in years:
    path = 'ch02/yob%d.txt' % year
    frame = pd.read_csv(path,names=columns,header=None)
    
    frame['year']=year
    pieces.append(frame)

#将所有数据整合到单个DataFrame中
names = pd.concat(pieces,ignore_index=True)
names.info()

In [ ]:
total_births = names.pivot_table(values ='births',columns='sex',index='year',aggfunc=sum)
total_births.tail()

In [ ]:
total_births.plot(kind='line',title='Total births by sex and year')

In [ ]:
def add_prop(group):
    #用于插入一个prop列，存放指定名字的婴儿数相对于总出生数的比例
    births = group.births.astype(float)
    group['prop'] = births/births.sum()
    return group

names = names.groupby(['year','sex']).apply(add_prop)
names.info()

In [ ]:
#有效性检查，验证prop综合是否为1，浮点型数据建议用numpy处理
import numpy as np
np.allclose(names.groupby(['year','sex']).prop.sum(),1)

In [ ]:
#取出每对SEX/YEAR组合的前1000个名字
def get_top1000(group):
    return group.sort_values('births',ascending=False)[:1000]

grouped = names.groupby(['year','sex'])
top1000 = grouped.apply(get_top1000)
top1000.info()

### 分析命名趋势  
pandas dataframe d[布尔型series]  
pandas dataframe d[[label list]]  
pandas dataframe d.plot()

In [ ]:
boys = top1000[top1000.sex == 'M']
girls = top1000[top1000['sex'] == 'F']
top1000=top1000.drop(['year','sex'],axis=1)
total_births = top1000.pivot_table(values='births',index='year',columns='name',aggfunc=sum)
total_births.info()

In [ ]:
subset=total_births[['John','Harry','Mary','Marilyn']]
subset.plot(subplots=True,figsize=(12,10),grid=False,title="Number of births per year")

##### 评估命名多样性的增长  
numpy Array常规创建 np.linspace()  
pandas series s.cumsum()  
pandas series s.searchsorted()  
pandas dataframe d.groupby()  
pandas series s.unstack()  

In [ ]:
table=top1000.pivot_table(values='prop',index='year',columns='sex',aggfunc=sum)
table.plot(title='Sum of table1000.prop by year and sex',yticks=np.linspace(0,1.2,13),xticks=range(1880,2020,10))

In [ ]:
#计算占总出生人数前50%的不同名字的数量
#只考虑2010年男孩名字
df = boys[boys.year == 2010]
df.info()

In [ ]:
"""
思路：
前50%,意味着需要排序
加起来够50%,意味着需要迭代
对于迭代，numpy有一种矢量实现
"""
#首先计算累积和
prop_cumsum = df.sort_values(by=['prop'],ascending=False).prop.cumsum()
prop_cumsum[:10]

In [ ]:
prop_cumsum.searchsorted(0.5)[0]+1

In [ ]:
df = boys[boys.year == 1900]
in1900 = df.sort_values(by=['prop'],ascending=False).prop.cumsum()
in1900.searchsorted(0.5)[0]+1

In [ ]:
def get_quantile_count(group,q=0.5):
    group = group.sort_values(by='prop',ascending=False)
    return group.prop.cumsum().searchsorted(q)[0]+1

diversity = top1000.groupby(['year','sex']).apply(get_quantile_count)
diversity = diversity.unstack('sex')
diversity.head()

In [ ]:
diversity.plot(title="Number of popular names in top 50%")

##### "最后一个字母"的变革
pandas series s.map()  
python 更多控制流工具 Lambda表达式  
pandas dataframe d.reindex()  
pandas dataframe d.sum()  
pandas dataframe 符号运算  
matplotlib pyplot p.subplots()  
matplotlib figure figsize  
matplotlib figure Figure画布  
matplotlib axes Axes坐标轴  
pandas series s.plot()  
pandas dataframe d.loc[]  
pandas dataframe d.T

In [ ]:
get_last_letter = lambda x:x[-1]
last_letters = names.name.map(get_last_letter)
last_letters.name='last_letter'
table = names.pivot_table(values='births',index=last_letters,columns=['sex','year'],aggfunc=sum)
subtable = table.reindex(columns=[1910,1960,2010],level='year')
subtable.head()

In [ ]:
subttl = subtable.sum(axis=0).astype(float)
subttl

In [ ]:
#男孩女孩名字中各个末字母的比例
letter_prop = subtable/subttl
import matplotlib.pyplot as plt
fig,axes = plt.subplots(nrows=2,ncols=1,figsize=(10,8))#先确定画布，再确定坐标轴
letter_prop['M'].plot(kind='bar',rot=0,ax=axes[0],title='Male')#在坐标轴上进行绘图
letter_prop['F'].plot(kind='bar',rot=0,ax=axes[1],title='Female',legend=False)#在坐标轴上进行绘图

In [ ]:
#可以看出，20世纪60年代开始，以字母“n"结尾的男孩名字出现了显著的增长
#按年度和性别对完整表进行规范化处理，并在男孩名字中选取几个字母，转置做成时间序列
letter_prop = table /table.sum().astype(float)
dny_ts = letter_prop.loc[['d','n','y'],'M'].T
dny_ts.head()

In [ ]:
#各年出生的男孩中名字以d/n/y结尾的人数比例
dny_ts.plot()

##### 变成女孩名字的男孩名字（以及相反的情况)
pandas series s.unique()  
numpy Array常规创建 np.array()  
pandas series s.isin()

In [ ]:
#找出其中以“lesl"开头的一组名字
all_names = top1000.name.unique()
#mask = np.array()
mask = np.array(['lesl' in x.lower() for x in all_names])
lesley_like = all_names[mask]
lesley_like

In [ ]:
#利用这个结果过滤其他的名字，并按名字分组计算出生数以查看相对频率
filtered = top1000[top1000.name.isin(lesley_like)]